In [1]:
from imutils.object_detection import non_max_suppression
import numpy as np
import pytesseract
import cv2
import sys
sys.path.append('./../template_matching')
from clean_image import clean_noise
from bounding_boxes_2 import find_orientation_threshold_rotate, remove_small_components, gauss_otsu_thresholding
#from bound_box_chars import find_bbox_chars

In [2]:
def show_wait_destroy_particular(winname, img):

    cv2.imshow(winname, img)   
    cv2.moveWindow(winname, 300, 0)
    cv2.waitKey(0)
    cv2.destroyWindow(winname)
    cv2.waitKey(1)

In [3]:
def decode_predictions(scores, geometry, min_confidence):
    # grab the number of rows and columns from the scores volume, then
    # initialize our set of bounding box rectangles and corresponding
    # confidence scores
    (numRows, numCols) = scores.shape[2:4]
    rects = []
    confidences = []

    # loop over the number of rows
    for y in range(0, numRows):
        # extract the scores (probabilities), followed by the
        # geometrical data used to derive potential bounding box
        # coordinates that surround text
        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]

        # loop over the number of columns
        for x in range(0, numCols):
            # if our score does not have sufficient probability,
            # ignore it
            if scoresData[x] < min_confidence:
                continue

            # compute the offset factor as our resulting feature
            # maps will be 4x smaller than the input image
            (offsetX, offsetY) = (x * 4.0, y * 4.0)

            # extract the rotation angle for the prediction and
            # then compute the sin and cosine
            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)

            # use the geometry volume to derive the width and height
            # of the bounding box
            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]

            # compute both the starting and ending (x, y)-coordinates
            # for the text prediction bounding box
            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)

            # add the bounding box coordinates and probability score
            # to our respective lists
            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])

    # return a tuple of the bounding boxes and associated confidences
    return (rects, confidences)

In [4]:
def find_text(img, params):
    min_confidence = params['min_confidence']
    width = params['width']
    height = params['height']
    padding = params['padding']
    psm = params['psm']
    east = 'frozen_east_text_detection.pb'

    image = img
    orig = image.copy()
    (origH, origW) = image.shape[:2]

    # set the new width and height and then determine the ratio in change
    # for both the width and height
    (newW, newH) = (width, height)
    rW = origW / float(newW)
    rH = origH / float(newH)

    # resize the image and grab the new image dimensions
    image = cv2.resize(image, (newW, newH))
    (H, W) = image.shape[:2]
    
    # define the two output layer names for the EAST detector model that
    # we are interested in -- the first is the output probabilities and the
    # second can be used to derive the bounding box coordinates of text
    layerNames = [
        "feature_fusion/Conv_7/Sigmoid",
        "feature_fusion/concat_3"]

    # load the pre-trained EAST text detector
    print("[INFO] loading EAST text detector...")
    net = cv2.dnn.readNet(east)
    
    # construct a blob from the image and then perform a forward pass of
    # the model to obtain the two output layer sets
    blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
        (0,0,0), swapRB=False, crop=True)
    net.setInput(blob)
    (scores, geometry) = net.forward(layerNames)

    # decode the predictions, then  apply non-maxima suppression to
    # suppress weak, overlapping bounding boxes
    (rects, confidences) = decode_predictions(scores, geometry, min_confidence)
    boxes = non_max_suppression(np.array(rects), probs=confidences)
    
    # initialize the list of results
    results = []

    # loop over the bounding boxes
    for (startX, startY, endX, endY) in boxes:
        # scale the bounding box coordinates based on the respective
        # ratios
        startX = int(startX * rW)
        startY = int(startY * rH)
        endX = int(endX * rW)
        endY = int(endY * rH)

        # in order to obtain a better OCR of the text we can potentially
        # apply a bit of padding surrounding the bounding box -- here we
        # are computing the deltas in both the x and y directions
        dX = int((endX - startX) * padding)
        dY = int((endY - startY) * padding)

        # apply padding to each side of the bounding box, respectively
        startX = max(0, startX - dX)
        startY = max(0, startY - dY)
        endX = min(origW, endX + (dX * 2))
        endY = min(origH, endY + (dY * 2))

        # extract the actual padded ROI
        roi = orig[startY:endY, startX:endX]

        # in order to apply Tesseract v4 to OCR text we must supply
        # (1) a language, (2) an OEM flag of 1, indicating that the we
        # wish to use the LSTM neural net model for OCR, and finally
        # (3) an OEM value, in this case, 7 which implies that we are
        # treating the ROI as a single line of text
        config = (f"-l eng --oem 1 -c tessedit_char_whitelist=01234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ.* --psm {psm}")
        text = pytesseract.image_to_string(roi, config=config)

        # add the bounding box coordinates and OCR'd text to the list
        # of results
        results.append(((startX, startY, endX, endY), text))
        
        # sort the results bounding box coordinates from top to bottom
    results = sorted(results, key=lambda r:r[0][1])

    # loop over the results
    for ((startX, startY, endX, endY), text) in results:
        # display the text OCR'd by Tesseract
        print("OCR TEXT")
        print("========")
        print("{}\n".format(text))

        # strip out non-ASCII text so we can draw the text on the image
        # using OpenCV, then draw the text and a bounding box surrounding
        # the text region of the input image
        text = "".join([c if ord(c) < 128 else "" for c in text]).strip()
        output = orig.copy()
        cv2.rectangle(output, (startX, startY), (endX, endY),
            (0, 0, 255), 2)
        cv2.putText(output, text, (startX, startY - 20),
            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

        # show the output image
        show_wait_destroy_particular("Text Detection", output)   

In [5]:
def convert_to_3channels(img):
    gray = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    return gray

In [6]:
# psm

#   0    Orientation and script detection (OSD) only.
#   1    Automatic page segmentation with OSD.
#   2    Automatic page segmentation, but no OSD, or OCR.
#   3    Fully automatic page segmentation, but no OSD. (Default)
#   4    Assume a single column of text of variable sizes.
#   5    Assume a single uniform block of vertically aligned text.
#   6    Assume a single uniform block of text.
#   7    Treat the image as a single text line.
#   8    Treat the image as a single word.
#   9    Treat the image as a single word in a circle.
#  10    Treat the image as a single character.
#  11    Sparse text. Find as much text as possible in no particular order.
#  12    Sparse text with OSD.
#  13    Raw line. Treat the image as a single text line,
# 			bypassing hacks that are Tesseract-specific.

In [32]:
params = {
    'min_confidence' : 0.97,
    'width' : 96*4,
    'height' : 32*4,
    'padding' : 0.1,
    'psm' : 8,
    'tessedit_write_images': True,
    'load_system_dawg': False,
    
}

In [33]:
def clean_noise2(img):
    #show_wait_destroy_particular('original',img)
    img = cv2.multiply(img, 1.2)
    img = cv2.fastNlMeansDenoising(img, None)
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.erode(img, kernel, iterations=20)
    #show_wait_destroy_particular('cleaned',img)
    return img

In [34]:
def add_border(img):
    # white
    top = int(0.2 * img.shape[0])  # shape[0] = rows
    bottom = top
    left = int(0.2 * img.shape[1])  # shape[1] = cols
    right = left
    dst = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, None, 255)
    
    # black
    top = int(0.05 * dst.shape[0])  # shape[0] = rows
    bottom = top
    left = int(0.05 * dst.shape[1])  # shape[1] = cols
    right = left
    dst = cv2.copyMakeBorder(dst, top, bottom, left, right, cv2.BORDER_CONSTANT, None, 0)
    show_wait_destroy_particular('boredr',dst)
    return dst

In [35]:
def process_image(img):
    img = gauss_otsu_thresholding(img)
    img = remove_small_components(img)
    img = add_border(img)
    img = convert_to_3channels(img)
    return img

In [14]:
numbers = list(range(0,5))
predictions = []
for num in numbers:

    src = cv2.imread(f'./../dataset-png-corrected/reg-cert-{num}.png', 0)
    print(src.shape)
    print('-'*40)
    print(num, end=' ')
    
    img_lic, img_date, img_vin =find_orientation_threshold_rotate(src)
    #print(img_lic, img_date, img_vin)
    #convert_to_3channels(img_lic)
    find_text(process_image(img_lic),params)
    #find_text(convert_to_3channels(remove_small_components(gauss_otsu_thresholding(img_date))), params)
    find_text(process_image(img_vin), params)
    
    #img = cv2.fastNlMeansDenoising(img_lic, None)
    #show_wait_destroy_particular('aa',img)



(3509, 2480)
----------------------------------------
0 Angle is 0.0
hl 0.7613178491592407
[INFO] loading EAST text detector...
OCR TEXT
T0820

OCR TEXT
HN

[INFO] loading EAST text detector...
OCR TEXT
EWFOAXXWPMAGBS4207

OCR TEXT
BARDTY

(3509, 2480)
----------------------------------------
1 Angle is 0.0
hl 1.0
[INFO] loading EAST text detector...
OCR TEXT
TOS4O

OCR TEXT
HE

[INFO] loading EAST text detector...
OCR TEXT
M1

OCR TEXT
EWFOAYYWDPMACCR0R

OCR TEXT
803873

(3509, 2480)
----------------------------------------
2 Angle is -90.0
vr 0.8407082557678223
[INFO] loading EAST text detector...
OCR TEXT
MX4100

OCR TEXT
O

[INFO] loading EAST text detector...
OCR TEXT
WDD208Z0R1F697282

OCR TEXT
STAY

OCR TEXT
ETYTSSETTYEY

(3509, 2480)
----------------------------------------
3 Angle is -90.0
vr 0.6287045478820801
[INFO] loading EAST text detector...
OCR TEXT
012

OCR TEXT
HM

[INFO] loading EAST text detector...
OCR TEXT
TITNTIA1

OCR TEXT
AISSO031U23A

(3509, 2480)
------------